In [230]:
import pandas as pd
def preprocess_header(df):
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.lower()
    return df

def fix_happiness_index(df):
    df.columns = df.columns.str.replace(" ", "_")


energy_df = pd.read_csv("Energy Consumption.csv")
energy_df = preprocess_header(energy_df)
energy_df_columns = ["country", "year", "total_consumption", "energy_consumption_per_capita"]
energy_df = energy_df[energy_df_columns]

happiness_df = pd.read_csv("Happiness Index.csv")
happiness_df = preprocess_header(happiness_df)
happiness_df["happiness_score"] = happiness_df["happiness_score"].str.replace(",", ".").astype(float)

air_quality_df = pd.read_csv("Air Quality.csv")
air_quality_df = preprocess_header(air_quality_df)
air_quality_df_columns = ["country", "year", "average_pm25_concentration"]
air_quality_df = air_quality_df[air_quality_df_columns]

gdp_df = pd.read_csv("GDP.csv")
gdp_df = preprocess_header(gdp_df)

unemployed_rate_df = pd.read_csv("Unemployed Rate.csv")
unemployed_rate_df = preprocess_header(unemployed_rate_df)

cpi_df = pd.read_csv("CPI.csv")
cpi_df = preprocess_header(cpi_df)

hdi_df = pd.read_csv("Human Development Index.csv")
hdi_df = preprocess_header(hdi_df)

max_year = air_pollution_df["year"].max()
energy_df_filtered = energy_df[energy_df["year"] <= max_year]
happiness_df_filtered = happiness_df[happiness_df["year"] <= max_year]

In [231]:
energy_df

,country,year,total_consumption,energy_consumption_per_capita
0,Australia,2015,5318.155,0.000223
1,Bangladesh,2015,2788.196,0.000018
2,Brazil,2015,10456.885,0.000051
3,Canada,2015,11215.963,0.000314
4,China,2015,76094.257,0.000055
...,...,...,...,...
235,Turkey,2022,6330.736,0.000074
236,United Arab Emirates,2022,4662.861,0.000494
237,United Kingdom,2022,6455.663,0.000096
238,United States,2022,58023.664,0.000172


In [232]:
gdp_df

,country,gdp,gdp_per_capita,year
0,Australia,1.140000e+12,47858.459864,2015
1,Bangladesh,5.520000e+11,3497.433948,2015
2,Brazil,3.020000e+12,14718.194722,2015
3,Canada,1.570000e+12,43938.059091,2015
4,China,1.830000e+13,13130.370278,2015
...,...,...,...,...
235,Turkey,2.390000e+12,28005.215016,2022
236,United Arab Emirates,7.800000e+11,82617.159076,2022
237,United Kingdom,2.620000e+12,38809.676989,2022
238,United States,1.950000e+13,57642.875345,2022


In [233]:
happiness_df_filtered["year"].unique()
happiness_df_filtered.head()

,country,happiness_score,year
0,Canada,7.427,2015
1,Netherlands,7.378,2015
2,Australia,7.284,2015
3,Mexico,7.187,2015
4,United States,7.119,2015


In [234]:
def calculate_aqi(concentration, breakpoints):
    for i in range(len(breakpoints)):
        if breakpoints[i][0] <= concentration <= breakpoints[i][1]:
            c_low, c_high = breakpoints[i][0], breakpoints[i][1]
            aqi_low, aqi_high = breakpoints[i][2], breakpoints[i][3]
            aqi = ((aqi_high - aqi_low) / (c_high - c_low)) * (concentration - c_low) + aqi_low
            return aqi
    return None

def aqi_category(aqi, breakpoints):
    for i in range(len(breakpoints)):
        if breakpoints[i][2] <= aqi <= breakpoints[i][3]:
            return breakpoints[i][4]
    return None

pm25_breakpoints = [
    (0, 9, 0, 50, "GOOD"), (9.1, 35.4, 51, 100, "MODERATE"), (35.5, 55.4, 101, 150, "UNHEALTHY FOR SENSITIVE"),
    (55.5, 125.4, 151, 200, "UNHEALTHY"), (125.5, 225.4, 201, 300, "VERY UNHEALTHY"), (225.5, 325.4, 301, 500, "HAZARDOUS"), (325.5, 99999.9, 501, 999, "VERY HAZARDOUS")
]

air_quality_df['aqi_pm25_concentration'] = air_quality_df['average_pm25_concentration'].apply(lambda x: calculate_aqi(x, pm25_breakpoints))
air_quality_df['aqi_pm25_category'] = air_quality_df['aqi_pm25_concentration'].apply(lambda x: aqi_category(x, pm25_breakpoints))

In [235]:
air_quality_df.head()

,country,year,average_pm25_concentration,aqi_pm25_concentration,aqi_pm25_category
0,Australia,2015,4.086677,22.703763,GOOD
1,Bangladesh,2015,78.139111,166.870049,UNHEALTHY
2,Brazil,2015,2.971552,16.508621,GOOD
3,Canada,2015,6.710247,37.279149,GOOD
4,China,2015,49.557086,135.612924,UNHEALTHY FOR SENSITIVE


In [236]:
gdp_df.head()

,country,gdp,gdp_per_capita,year
0,Australia,1.140000e+12,47858.459864,2015
1,Bangladesh,5.520000e+11,3497.433948,2015
2,Brazil,3.020000e+12,14718.194722,2015
3,Canada,1.570000e+12,43938.059091,2015
4,China,1.830000e+13,13130.370278,2015


In [237]:
unemployed_rate_df.head()

,country,unemployed_rate,year
0,United Arab Emirates,1.792,2015
1,Australia,6.055,2015
2,Bangladesh,4.382,2015
3,Brazil,8.538,2015
4,Canada,6.945,2015


In [238]:
cpi_df.head()

,country,cpi,year
0,United Arab Emirates,4.069966,2015
1,Australia,1.508367,2015
2,Bangladesh,6.194280,2015
3,Brazil,9.029901,2015
4,Canada,1.125241,2015


In [239]:
hdi_df.head()

,country,hdi,year
0,United Arab Emirates,0.860,2015
1,Australia,0.933,2015
2,Bangladesh,0.604,2015
3,Brazil,0.752,2015
4,Canada,0.927,2015


In [240]:
tasks_columns = ["country", "year", "total_consumption", "energy_consumption_per_capita", "happiness_score", "average_pm25_concentration", "aqi_pm25_concentration", "aqi_pm25_category", "gdp", "gdp_per_capita", "unemployed_rate", "cpi", "hdi"]
tasks_df = pd.merge(gdp_df, happiness_df, on=["country", "year"])
tasks_df = pd.merge(tasks_df, energy_df, on=["country", "year"])
tasks_df = pd.merge(air_quality_df, tasks_df, on=["country", "year"])
tasks_df = pd.merge(unemployed_rate_df, tasks_df, on=["country", "year"])
tasks_df = pd.merge(cpi_df, tasks_df, on=["country", "year"])
tasks_df = pd.merge(hdi_df, tasks_df, on=["country", "year"])
tasks_df = tasks_df[tasks_columns]
tasks_df = tasks_df[(tasks_df != 0).all(axis=1)]

In [186]:
tasks_df.describe()

,year,total_consumption,energy_consumption_per_capita,happiness_score,average_pm25_concentration,aqi_pm25_concentration,gdp,gdp_per_capita,unemployed_rate,cpi,hdi
count,133.000000,133.000000,133.000000,133.000000,133.000000,131.000000,1.330000e+02,133.000000,133.000000,133.000000,133.000000
mean,2017.661654,13208.656143,0.000120,6.123479,17.296227,59.622850,3.895564e+12,29990.250767,6.290233,2.810248,0.833474
std,1.886504,19086.289162,0.000098,0.912153,18.646740,38.600692,5.489587e+12,17843.690594,3.918791,3.114159,0.109957
min,2015.000000,2022.000000,0.000014,3.573300,2.674286,14.857143,5.520000e+11,3497.433948,0.597000,-1.931081,0.535000
25%,2016.000000,4652.083000,0.000054,5.345000,5.535363,30.727352,1.150000e+12,14321.971781,3.746000,0.989487,0.757000
50%,2018.000000,6488.798000,0.000103,6.310000,8.339847,44.885306,1.960000e+12,32922.115313,4.830000,1.911401,0.889000
75%,2019.000000,10262.278000,0.000160,6.939600,23.215000,77.421806,3.050000e+12,44228.793134,8.010000,3.520257,0.926000
max,2022.000000,90849.856000,0.000490,7.488000,90.244225,175.355751,2.420000e+13,79790.178969,22.058000,16.332464,0.951000


In [187]:
tasks_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, 1 to 146
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   country                        133 non-null    object 
 1   year                           133 non-null    int64  
 2   total_consumption              133 non-null    float64
 3   energy_consumption_per_capita  133 non-null    float64
 4   happiness_score                133 non-null    float64
 5   average_pm25_concentration     133 non-null    float64
 6   aqi_pm25_concentration         131 non-null    float64
 7   aqi_pm25_category              131 non-null    object 
 8   gdp                            133 non-null    float64
 9   gdp_per_capita                 133 non-null    float64
 10  unemployed_rate                133 non-null    float64
 11  cpi                            133 non-null    float64
 12  hdi                            133 non-null    float64


In [188]:
tasks_df.isnull().sum() 

country                          0
year                             0
total_consumption                0
energy_consumption_per_capita    0
happiness_score                  0
average_pm25_concentration       0
aqi_pm25_concentration           2
aqi_pm25_category                2
gdp                              0
gdp_per_capita                   0
unemployed_rate                  0
cpi                              0
hdi                              0
dtype: int64

In [189]:
tasks_df.dropna(inplace=True)

In [193]:
tasks_df.isnull().sum() 

country                          0
year                             0
total_consumption                0
energy_consumption_per_capita    0
happiness_score                  0
average_pm25_concentration       0
aqi_pm25_concentration           0
aqi_pm25_category                0
gdp                              0
gdp_per_capita                   0
unemployed_rate                  0
cpi                              0
hdi                              0
dtype: int64

In [190]:
tasks_df

,country,year,total_consumption,energy_consumption_per_capita,happiness_score,average_pm25_concentration,aqi_pm25_concentration,aqi_pm25_category,gdp,gdp_per_capita,unemployed_rate,cpi,hdi
1,Australia,2015,5318.155,0.000223,7.284,4.086677,22.703763,GOOD,1.140000e+12,47858.459864,6.055,1.508367,0.933
2,Bangladesh,2015,2788.196,0.000018,4.694,78.139111,166.870049,UNHEALTHY,5.520000e+11,3497.433948,4.382,6.194280,0.604
3,Brazil,2015,10456.885,0.000051,6.983,2.971552,16.508621,GOOD,3.020000e+12,14718.194722,8.538,9.029901,0.752
4,Canada,2015,11215.963,0.000314,7.427,6.710247,37.279149,GOOD,1.570000e+12,43938.059091,6.945,1.125241,0.927
5,China,2015,76094.257,0.000055,5.140,49.557086,135.612924,UNHEALTHY FOR SENSITIVE,1.830000e+13,13130.370278,4.650,1.437024,0.741
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,Italy,2021,5886.828,0.000099,6.483,7.373092,40.961621,GOOD,2.060000e+12,34773.604255,9.497,1.873783,0.899
142,Saudi Arabia,2021,7982.646,0.000222,6.494,3.507692,19.487179,GOOD,1.680000e+12,46731.056871,6.621,3.063290,0.867
143,Thailand,2021,4894.876,0.000068,5.985,20.668848,72.554127,MODERATE,1.100000e+12,15362.891611,1.215,1.230395,0.797
144,United States,2021,56660.668,0.000168,6.951,6.938957,38.549760,GOOD,1.910000e+13,56676.956116,5.349,4.697859,0.921


In [191]:
tasks_df[tasks_df["country"] == "China"]

,country,year,total_consumption,energy_consumption_per_capita,happiness_score,average_pm25_concentration,aqi_pm25_concentration,aqi_pm25_category,gdp,gdp_per_capita,unemployed_rate,cpi,hdi
5,China,2015,76094.257,0.000055,5.1400,49.557086,135.612924,UNHEALTHY FOR SENSITIVE,1.830000e+13,13130.370278,4.65,1.437024,0.741
29,China,2016,76707.739,0.000055,5.2450,45.739159,126.211999,UNHEALTHY FOR SENSITIVE,1.950000e+13,13909.796542,4.56,2.000002,0.749
53,China,2017,79820.423,0.000057,5.2730,44.610489,123.432863,UNHEALTHY FOR SENSITIVE,2.090000e+13,14819.794476,4.47,1.593136,0.757
76,China,2018,83777.827,0.000059,5.2460,40.153056,112.457274,UNHEALTHY FOR SENSITIVE,2.230000e+13,15736.702359,4.31,2.074790,0.766
99,China,2019,87805.094,0.000062,5.1910,38.475538,108.326701,UNHEALTHY FOR SENSITIVE,2.360000e+13,16597.929857,4.56,2.899234,0.775
120,China,2020,90849.856,0.000064,5.1239,33.631841,96.705712,MODERATE,2.420000e+13,16983.292886,5.00,2.419422,0.781


In [192]:
tasks_df[tasks_df["country"] == "United States"]

,country,year,total_consumption,energy_consumption_per_capita,happiness_score,average_pm25_concentration,aqi_pm25_concentration,aqi_pm25_category,gdp,gdp_per_capita,unemployed_rate,cpi,hdi
23,United States,2015,55432.831,0.000171,7.1190,7.236025,40.200137,GOOD,1.690000e+13,52062.831668,5.280,0.118627,0.924
47,United States,2016,55601.287,0.000170,7.1040,6.506025,36.144582,GOOD,1.720000e+13,52565.597220,4.869,1.261583,0.926
69,United States,2017,56009.588,0.000170,6.9930,6.878247,38.212483,GOOD,1.760000e+13,53367.094975,4.355,2.130110,0.928
92,United States,2018,57955.215,0.000174,6.8860,7.011235,38.951303,GOOD,1.810000e+13,54495.087181,3.896,2.442583,0.930
116,United States,2019,57651.207,0.000172,6.8920,6.493157,36.073097,GOOD,1.850000e+13,55336.257800,3.669,1.812210,0.933
133,United States,2020,53855.538,0.000160,6.9396,6.744562,37.469787,GOOD,1.800000e+13,53580.675065,8.055,1.233584,0.923
144,United States,2021,56660.668,0.000168,6.9510,6.938957,38.549760,GOOD,1.910000e+13,56676.956116,5.349,4.697859,0.921


In [181]:
tasks_df[tasks_df["country"] == "Japan"]

,country,year,total_consumption,energy_consumption_per_capita,happiness_score,average_pm25_concentration,aqi_pm25_concentration,aqi_pm25_category,gdp,gdp_per_capita,unemployed_rate,cpi,hdi
84,Japan,2018,13132.519,0.000104,5.915,11.471067,55.417577,MODERATE,4.850000e+12,38414.054912,2.467,0.989095,0.917
107,Japan,2019,12913.366,0.000103,5.886,10.107067,52.876284,MODERATE,4.830000e+12,38396.816069,2.351,0.468776,0.918


In [195]:
tasks_df.to_csv("tasks.csv", index=False)

## Correlation Calculation for Node-link

In [ ]:
from scipy.stats import pearsonr
tasks_df.dropna(inplace=True)
background = ["total_consumption","energy_consumption_per_capita", "happiness_score"]
sub_factor = ["aqi_pm25_concentration", "gdp_per_capita", "gdp", "unemployed_rate", "cpi", "hdi"]
factor = ["environment", "finance", "living_quality"]

pearsonr_df = pd.DataFrame(columns=["background", "aqi_pm25_concentration", "gdp_per_capita", "gdp", "unemployed_rate", "cpi", "hdi"])

# Iterate over each combination of background and sub_factor
i = 0
for bg in background:
    sub_factor_pearsonr = []
    for sf in sub_factor:
        yearly_correlations = []
        for year in tasks_df["year"].unique():
            list1 = tasks_df[tasks_df["year"] == year][bg]
            list2 = tasks_df[tasks_df["year"] == year][sf]
            if len(list1) > 1 and len(list2) > 1:  
                corr, _ = pearsonr(list1, list2)
                yearly_correlations.append(corr)
        if yearly_correlations:
            avg_corr = round(sum(yearly_correlations) / len(yearly_correlations), 3)
        else:
            avg_corr = None
        sub_factor_pearsonr.append(avg_corr)
        print(f"{bg} and {sf} average correlation: {avg_corr}")
    pearsonr_df.loc[i] = [bg] + sub_factor_pearsonr
    i += 1


pearsonr_df.set_index("background", inplace=True)



total_consumption and aqi_pm25_concentration average correlation: 0.163
total_consumption and gdp_per_capita average correlation: 0.057
total_consumption and gdp average correlation: 0.983
total_consumption and unemployed_rate average correlation: -0.102
total_consumption and cpi average correlation: 0.016
total_consumption and hdi average correlation: 0.015
energy_consumption_per_capita and aqi_pm25_concentration average correlation: -0.346
energy_consumption_per_capita and gdp_per_capita average correlation: 0.867
energy_consumption_per_capita and gdp average correlation: -0.114
energy_consumption_per_capita and unemployed_rate average correlation: -0.059
energy_consumption_per_capita and cpi average correlation: -0.382
energy_consumption_per_capita and hdi average correlation: 0.685
happiness_score and aqi_pm25_concentration average correlation: -0.703
happiness_score and gdp_per_capita average correlation: 0.782
happiness_score and gdp average correlation: -0.067
happiness_score an

In [224]:
pearsonr_df

,aqi_pm25_concentration,gdp_per_capita,gdp,unemployed_rate,cpi,hdi,finance,environment,living_quality
background,,,,,,,,,
total_consumption,0.163,0.057,0.983,-0.102,0.016,0.015,0.458,0.163,0.015
energy_consumption_per_capita,-0.346,0.867,-0.114,-0.059,-0.382,0.685,0.263,-0.346,0.685
happiness_score,-0.703,0.782,-0.067,0.018,-0.303,0.821,0.223,-0.703,0.821


In [225]:
pearsonr_df.to_csv("pearsonr_df.csv", index=False)

### rubbish bin :)

In [ ]:
# old version
from scipy.stats import pearsonr

# Define the features for each dataset
energy_features_for_task_six = ["country", "year", "total_consumption"]
happiness_features_for_task_six = ["country", "year", "happiness_score"]
air_pollution_features_for_task_six = ["country", "year", "aqi_pm25_concentration", "aqi_pm25_category"]
gdp_df_features_for_task_six = ["country", "year", "gdp_per_capita", "gdp"]

task_six_df = pd.merge(energy_df_filtered[energy_features_for_task_six], happiness_df_filtered[happiness_features_for_task_six], on=["country", "year"])
task_six_df = pd.merge(task_six_df, air_quality_df[air_pollution_features_for_task_six], on=["country", "year"])
task_six_df = pd.merge(task_six_df, gdp_df[gdp_df_features_for_task_six], on=["country", "year"])
task_six_df = pd.merge(task_six_df, unemployed_rate_df, on=["country", "year"])
task_six_df = pd.merge(task_six_df, cpi_df, on=["country", "year"])
task_six_df = pd.merge(task_six_df, hdi_df, on=["country", "year"])

task_six_df.head()
background = ["total_consumption", "happiness_score"]
sub_factor = ["aqi_pm25_concentration", "gdp_per_capita", "gdp", "unemployed_rate", "cpi", "hdi"]
factor = ["environment", "finance", "living_quality"]

pearsonr_df = pd.DataFrame(columns=["background", "aqi_pm25_concentration", "gdp_per_capita", "gdp", "unemployed_rate", "cpi", "hdi"])

# calculate the correlation between each background and sub_factor
i = 0
for bg in background:
    sub_factor_pearsonr = []
    for sf in sub_factor:
        list1 = task_six_df[bg]
        list2 = task_six_df[sf]
        corr, _ = pearsonr(list1, list2)
        corr = round(corr, 3)
        sub_factor_pearsonr.append(corr)
        print(f"{bg} and {sf} correlation: {corr:.3f}")
    pearsonr_df.loc[i] = [bg] + sub_factor_pearsonr
    i += 1


pearsonr_df.set_index("background", inplace=True)

# Calculate the average correlation for each factor
environment = ["aqi_pm25_concentration"]
living_quality = ["hdi"]

for bg in background:
    task_six_df_cp = task_six_df.copy()
    if bg == "total_consumption":
        finance = ["gdp", "cpi", "unemployed_rate"]
    else:
        finance = ["gdp_per_capita", "cpi", "unemployed_rate"]
    scaler = StandardScaler()
    tasks_df[finance] = scaler.fit_transform(tasks_df[finance])
    tasks_df['finance'] = tasks_df[finance].mean(axis=1)
    corr, _ = pearsonr(tasks_df['finance'], tasks_df[bg])
    pearsonr_df.loc[bg, "finance"] = round(corr, 3)
    
pearsonr_df['environment'] = pearsonr_df["aqi_pm25_concentration"]
pearsonr_df['living_quality'] = pearsonr_df["hdi"]


pearsonr_df

,aqi_pm25_concentration,gdp_per_capita,gdp,unemployed_rate,cpi,human_development_index
background,,,,,,
total_consumption,0.201,0.004,0.979,-0.088,-0.070,-0.006
happiness_score,-0.635,0.751,-0.116,-0.043,-0.436,0.797


In [ ]:
# byebye
def scale_series(series):
    return 100 * (series / series.max())

scaled_df = task_three_df.copy()
for column in task_three_df.select_dtypes(include=['float64']).columns:
    scaled_df[f"indexed_{column}"] = scale_series(task_three_df[column])
    
scaled_df.to_csv("task_three_scaled.csv", index=False)

scaled_df.head()

,country,year,total_consumption,happiness_score,aqi_pm25_concentration,aqi_pm25_category,indexed_total_consumption,indexed_happiness_score,indexed_aqi_pm25_concentration
0,Australia,2015,3303.155,7.284,22.703763,GOOD,3.718519,97.275641,12.947259
1,Bangladesh,2015,773.196,4.694,166.870049,UNHEALTHY,0.870424,62.686966,95.160865
2,Brazil,2015,8441.885,6.983,16.508621,GOOD,9.503432,93.255876,9.414359
3,Canada,2015,9200.963,7.427,37.279149,GOOD,10.357962,99.185363,21.259154
4,China,2015,74079.257,5.140,135.612924,UNHEALTHY FOR SENSITIVE,83.394548,68.643162,77.335886


In [87]:
scaled_df[scaled_df["country"] == "United States"]

,country,year,total_consumption,happiness_score,aqi_pm25_concentration,aqi_pm25_category,indexed_total_consumption,indexed_happiness_score,indexed_aqi_pm25_concentration
23,United States,2015,53417.831,7.1190,40.200137,GOOD,60.134997,95.072115,22.924904
47,United States,2016,53585.287,7.1040,36.144582,GOOD,60.323510,94.871795,20.612145
69,United States,2017,53992.588,6.9930,38.212483,GOOD,60.782028,93.389423,21.791405
92,United States,2018,55937.215,6.8860,38.951303,GOOD,62.971187,91.960470,22.212732
116,United States,2019,55632.207,6.8920,36.073097,GOOD,62.627825,92.040598,20.571379
133,United States,2020,51835.538,6.9396,37.469787,GOOD,58.353734,92.676282,21.367869
144,United States,2021,54639.668,6.9510,38.549760,GOOD,61.510477,92.828526,21.983744
